# Training CNN on multiple images under varying cloud cover %s

In [1]:
# import ee
from IPython import display
# import math
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
# from osgeo import gdal
# import tempfile
import tensorflow as tf
# import urllib
import rasterio
from zipfile import *
import math
from math import sqrt
import sklearn
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, accuracy_score
import time

### Functions to stacking tifs, preprocessing, splitting data into train/val, and training CNN

In [2]:
def tifStacker(path, img, feat_list_new, overwrite=False): 

    """ Reorders the tifs (i.e. individual bands) downloaded from GEE according to feature order in feat_list_new, 
    then stacks them all into one multiband image called 'stack.tif' located in input path. Requires rasterio, 
    os, from zipfile import *
    
    Ideally want to have this function in another notebook and call it, but running into problems - ZipFile not found 
    
    from ipynb.fs.full.useful_funcs import tifStacker
    
    Parameters
    ----------
    path : str 
        Path to image folder
    img :str 
        Name of image file (without file extension)
    feat_list_new : list
        List of feature names (str) to be the desired order of the output stacked .tif - target feature must be last
    
    Returns
    ----------
    "stacked.tif" in 'path' location 
    feat_list_files : list 
        Not sure what that is or what it's for 
    
    """
    
    file_list = []
    path = path+'images/'+img
    
    # This gets the name of all files in the zip folder, and formats them into a full path readable by rasterio.open()
    with ZipFile(path + '/' + img + '.zip', 'r') as f:
        names = f.namelist()
        names = ['zip://'+ path + '/' + img + '.zip!' +name for name in names]
        for file in names:
            if file.endswith('.tif'):
                file_list.append(file)
    
    feat_list_files = list(map(lambda x: x.split('.')[-2], file_list)) # Grabs a list of features in file order        
    
    if overwrite=True:
        # Remove stack file if already exists
        try:
            os.remove(path + '/stack/stack.tif')
            print('Removing existing "stack.tif" and creating new one')
        except FileNotFoundError:
            print('No existing "stack.tif". Creating "stack.tif"')
            
        # Create 1 row df of file names where each col is a feature name, in the order files are stored locally
        file_arr = pd.DataFrame(data=[file_list], columns=feat_list_files)

        # Then index the file list by the ordered list of feature names used in training
        file_arr = file_arr.loc[:, feat_list_new]

        # The take this re-ordered row as a list - the new file_list
        file_list = list(file_arr.iloc[0,:])

        # Read metadata of first file. This needs to be a band in float32 dtype, because it sets the metadata for the entire stack
        # and we are converting the other bands to float64
        with rasterio.open(file_list[1]) as src0:
            meta = src0.meta
            meta['dtype'] = 'float32'
        #         print(meta)

        # Update meta to reflect the number of layers
        meta.update(count = len(file_list))

        # Read each layer, convert to float, and write it to stack
        # There's also a gdal way to do this, but unsure how to convert to float: https://gis.stackexchange.com/questions/223910/using-rasterio-or-gdal-to-stack-multiple-bands-without-using-subprocess-commands

        # Make new directory for stacked tif if it doesn't already exist
        try:
            os.mkdir(path +'/stack')
        except FileExistsError:
            print('Stack directory already exists') 
            
        with rasterio.open(path + '/stack/stack.tif', 'w', **meta) as dst:
            for id, layer in enumerate(file_list, start=0):
                with rasterio.open(layer) as src1:
                    dst.write_band(id+1, src1.read(1).astype('float32'))
                    
    else:
        print('"Stack.tif" already exists')

    return feat_list_files

This loads the stacked image and masks it 9 times for cloud cover 10-90%, resulting in 9 arrays and 9 tuples of row, col indices of the array cells with non-nan values. It might take too much memory to have all 9 arrays loaded though ...

In [3]:
def preprocessing(path, img, pctl):
    """
    Masks stacked image with cloudmask by converting cloudy values to NaN
    
    Parameters
    ----------
    path : str 
        Path to image folder
    img :str 
        Name of image file (without file extension)
    pctl : list of int
        List of integers of cloud cover percentages to mask image with (10, 20, 30, etc.)
    
    Returns
    ----------
    data : array
        3D array identical to input stacked image but with cloudy pixels masked
    data_ind : list?
        List? of indices in 'data' where cloudy pixels were masked. Used for reconstructing the image later 
    """
    
    # Get local image
    with rasterio.open(path + 'images/'+ img + '/stack/stack.tif', 'r') as ds:
        data = ds.read()
        data = data.transpose((1, -1, 0)) # Not sure why the rasterio.read output is originally (D, W, H)
    
    # load cloudmasks
    cloudMaskDir = path+'clouds'
    
    cloudMask = np.load(cloudMaskDir+'/'+img+'_clouds.npy')
    cloudMask = cloudMask < np.percentile(cloudMask, pctl)
    
    # Need to remove NaNs because any arithmetic operation involving an NaN will result in NaN
    data[cloudMask] = -999999
    
    # Convert -999999 to None
    data[data == -999999] = np.nan

    # Get indices of non-nan values. These are the indices of the original image array
    data_ind = np.where(~np.isnan(data[:,:,1]))
        
    return data, data_ind

In [4]:
def trainVal(data):
    
    HOLDOUT_FRACTION = 0.1

    # Reshape into a single vector of pixels.
    data_vector = data.reshape([data.shape[0] * data.shape[1], data.shape[2]])

    # Remove NaNs
    data_vector = data_vector[~np.isnan(data_vector).any(axis=1)]
    data_vector.shape

    # Select only the valid data and shuffle it.
    # valid_data = data_vector[numpy.equal(data_vector[:,8], 1)]
    # np.random.shuffle(data_vector)

    # Hold out a fraction of the labeled data for validation.
    training_size = int(data_vector.shape[0] * (1 - HOLDOUT_FRACTION))
    training_data = data_vector[0:training_size,:]
    validation_data = data_vector[training_size:-1,:]

    # Compute per-band means and standard deviations of the input bands.
    data_mean = training_data[:,0:14].mean(0)
    data_std = training_data[:,0:14].std(0)
    
    return [data_vector, training_data, validation_data, data_mean, data_std, training_size]

In [8]:
# CNN layer builder
# Function to train CNN on image, save model, and return performance metrics
def make_nn_layer(input, output_size):
    input_size = input.get_shape().as_list()[1]
    weights = tf.Variable(tf.truncated_normal(
        [input_size, output_size],
        stddev=1.0 / math.sqrt(float(input_size))))
    biases = tf.Variable(tf.zeros([output_size]))
    return tf.matmul(input, weights) + biases


In [9]:
def CNNtrainer(data_vector, training_data, validation_data, data_mean, data_std, model_path, img, pctl):

    model_path = model_path+img+'/'+img+'_clouds_'+str(pctl)
    model_name = img+'_clouds_'+str(pctl)
    checkpoint_filename = model_name+'_checkpoint'
    
    # Make a new directory for the model
    try:
        os.mkdir(model_path)
    except FileExistsError:
        print('Model directory already exists')
        
#     tf.reset_default_graph()
    
    import time
    def timer(start,end, formatted = True):
        if formatted == True: # Returns full formated time in hours, minutes, seconds
            hours, rem = divmod(end-start, 3600)
            minutes, seconds = divmod(rem, 60)
            return str("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
        else: # Returns minutes + fraction of minute
            minutes, seconds = divmod(time.time() - start_time, 60)
            seconds = seconds/60
            minutes = minutes + seconds
            return str(minutes)

    # Had to alter some config and runoptions because kept running into OOM at last step during eval 
    config = tf.ConfigProto(
        device_count = {'GPU': 0}
    )
    config.gpu_options.allow_growth = True
    run_options=tf.RunOptions(report_tensor_allocations_upon_oom=True)

    flooded = feat_list_files.index('flooded')
    
    # ------------------------------------------------------------------------------------------------
    NUM_INPUT_BANDS = 14
    NUM_HIDDEN_1 = 15
    NUM_HIDDEN_2 = 15
    NUM_CLASSES = 2
    BATCH_SIZE = 1000
    NUM_BATCHES = 1000

    input = tf.placeholder(tf.float32, shape=[None, NUM_INPUT_BANDS], name='input')
    labels = tf.placeholder(tf.float32, shape=[None], name='labels')
    
    normalized = (input - data_mean) / data_std
    hidden1 = tf.nn.tanh(make_nn_layer(normalized, NUM_HIDDEN_1), name='hidden1')
    hidden2 = tf.nn.tanh(make_nn_layer(hidden1, NUM_HIDDEN_2), name='hidden2')
    logits = make_nn_layer(hidden2, NUM_CLASSES)
    outputs = tf.argmax(logits, 1, name='outputs')

    int_labels = tf.to_int64(labels)
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits = logits, labels = int_labels, name='xentropy')
    train_step = tf.train.AdamOptimizer().minimize(cross_entropy) # should we minimize something else?

    correct_prediction = tf.equal(outputs, int_labels, name='correct_prediction')
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')
    # ------------------------------------------------------------------------------------------------
    
    mySaver = tf.train.Saver(save_relative_paths=True)
    init_op = tf.global_variables_initializer()
    start_time = time.time()
    
    with tf.Session(config=config) as sess:
        
        sess.run(init_op, options=run_options)

        training_dict = {
            input: training_data[:,0:14],
            labels: training_data[:,14],
        }

        validation_dict = {
            input: validation_data[:,0:14],
            labels: validation_data[:,14],
        }

        for i in range(NUM_BATCHES):
            batch = training_data[np.random.choice(training_size, BATCH_SIZE, False),:]
            train_step.run({input: batch[:,0:14], labels: batch[:,14]})

            if i % 100 == 0 or i == NUM_BATCHES - 1:
    #             print('Train acc. %.2f%%, val acc. %.2f%%, train recall %.2f%, val recall %.2f%, train precision %.2f%, val precision %.2f%, at step %d' 
                print('Train acc. %.2f%%, val acc. %.2f%%, at step %d' 
                      % (accuracy.eval(training_dict) * 100,
                         accuracy.eval(validation_dict) * 100, 
                         i))
                
        output_data = outputs.eval({input: data_vector[:,0:14]})

        # Save the model
        mySaver.save(sess, model_path+'/'+model_name+'.ckpt', 
                    global_step = NUM_BATCHES)
#                     latest_filename=checkpoint_filename)

    print('CNN training runtime for ' + str(pctl) + '% cloud cover: ' + timer(start_time, time.time()))
    
    return output_data


### Running on multiple images

In [10]:
#===================================
# Get list of images to train CNN on

import os

path = 'C:/Users/ipdavies/CPR/data/'

# Get list of all images
img_list = []
for file in os.listdir(path+'images'):
        img_list.append(file)

print(img_list)

# Manually creating list of images
img_list = ['4101_LC08_027038_20131103', '4101_LC08_027038_20131103_2', '4101_LC08_027039_20131103',
           '4115_LC08_021033_20131227_1', '4115_LC08_021033_20131227_2']

['4101_LC08_027038_20131103', '4101_LC08_027038_20131103_2', '4101_LC08_027039_20131103', '4115_LC08_021033_20131227_1', '4115_LC08_021033_20131227_2', '4337_LC08_026038_20160325_1', 'clear_4337_LC08_026038_20160325_1', 'clouds_4337_LC08_026038_20160325_1']


In [15]:
img_list = ['4101_LC08_027038_20131103_1']

### Generate cloud cover

In [18]:
from cloud_generation import cloudGenerator

ModuleNotFoundError: No module named 'cloud_generation'

In [16]:
# Run all of the functions: tif stack, preprocessing, CNN training, model and validation metric export

path = 'C:/Users/ipdavies/CPR/data/'

# Order in which features should be stacked to create stacked tif
feat_list_new = ['aspect','curve', 'developed', 'distExtent', 'elevation', 'forest',
 'GSW_maxExtent', 'hand', 'other_landcover', 'planted', 'slope', 'spi', 'twi', 'wetlands', 'flooded']

# Directory with trained CNNs
model_path = path+'models/cnn_vary_clouds/'

# Cloud cover %s to use
pctls = [10,20,30,40,50,60,70,80,90]

for j, img in enumerate(img_list):

    #  Stack all the flood imagery
    feat_list_files = tifStacker(path, img, feat_list_new)

    #  Stack all the flood imagery
    feat_list_files = tifStacker(path, img, feat_list_new)

    precision = []
    recall = []
    f1 = []
    accuracy = []
    times = []

    # Load stacked image, preprocess
    for i, pctl in enumerate(pctls):

        data, data_ind = preprocessing(path, img, pctl)

        data_vector, training_data, validation_data, data_mean, data_std, training_size = trainVal(data)

        start_time = time.time() # Start timer for CNN training

        print(img)
        
        y_pred = CNNtrainer(data_vector, training_data, validation_data, data_mean, data_std, model_path, img, pctl)

        times.append(timer(start_time, time.time(), False)) # Elapsed time in minutes

        y_true = data_vector[:,14]

        precision.append(sklearn.metrics.precision_score(y_true, y_pred))
        recall.append(sklearn.metrics.recall_score(y_true, y_pred))
        f1.append(sklearn.metrics.f1_score(y_true, y_pred))
        accuracy.append(sklearn.metrics.accuracy_score(y_true, y_pred))

    valMetrics = pd.DataFrame(np.column_stack([pctls, accuracy, precision, recall, f1, times]),
                          columns=['cloud_cover','accuracy','precision','recall','f1', 'time'])
    
    valMetrics.to_csv(model_path+img+'/valMetrics.csv', index=False)

    valMetrics.plot(x='cloud_cover', y=['recall', 'precision','f1','accuracy'])


Creating "stack.tif"


C:\Users\ipdavies\AppData\Local\Continuum\miniconda3\envs\tensorflow\lib\site-packages\rasterio\__init__.py:160: FutureWarning: GDAL-style transforms are deprecated and will not be supported in Rasterio 1.0.
  transform = guard_transform(transform)


Stack directory already exists
Removing existing "stack.tif" and creating new one


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/ipdavies/CPR/data/clouds/4101_LC08_027038_20131103_1_clouds.npy'